# <연애 상담 챗봇>

## 1. 모델/데이터 불러오기 

In [2]:
# !pip install sentence_transformers

import torch
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# SBERT 모델 불러오기
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# 학습된 input, output df 불러오기
data = pd.read_csv('/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_자연어처리프로젝트/application_love_chatbot_sbert_v1/data/csv/감성대화_청년_연결출_9000개')
data = data[['input','output']]

# input별 embedding data 불러오기
embedding_data = torch.load('/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_자연어처리프로젝트/application_love_chatbot_sbert_v1/data/embedding_data/love_embedding_data.pt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 10.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=ac6a45acc1734a160596aa363ec757d34cb52857bed80aa726b5d62b78c8f74c
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


## 2. 챗봇 대화 함수 선언

In [4]:
last_sentence = ''
last_sim_idx_list = []
overlap_cnt = 0

def talk_to_bot(sentence):

  global last_sentence 
  global last_sim_idx_list
  global overlap_cnt

  overlap_thresholds = 0.85
  remove_sim_idx = None

  sentence = sentence.replace(" ","")
  sentence_encode = model.encode(sentence)
  sentence_tensor = torch.tensor(sentence_encode)

  last_sentence = last_sentence.replace(" ","")
  last_sentence_encode = model.encode(last_sentence)
  last_sentence_tensor = torch.tensor(last_sentence_encode)

  overlap_cos_sim = util.cos_sim(sentence_tensor, last_sentence_tensor)

  cos_sim = util.cos_sim(sentence_tensor, embedding_data)

  # 이전 질문과 문장이 0.85이상 유사하면 
  # 해당 질문-답변 출력 X
  # 총 3번까지

  if overlap_cos_sim >= overlap_thresholds and overlap_cnt < 3 :
    print('************************ 이전 질문과 유사함 ************************')
    remove_sim_idx_list = last_sim_idx_list

    for i in remove_sim_idx_list:
      cos_sim[0][int(i)] = 0
    overlap_cnt += 1
    print(f'이전 출력 idx: {last_sim_idx_list}')
    print(f'이전 질문과 유사도: {overlap_cos_sim}')
    print(f'중복횟수 : {overlap_cnt}')
  else:
    overlap_cnt = 0
    last_sim_idx_list = []

  print(f'사용자의 질문: {sentence} \n')

  dot_score = util.dot_score(sentence_tensor, embedding_data)
  ascending_cos_sim = np.sort(cos_sim)
  descending_cos_sim = ascending_cos_sim[::-1]
  # print(f'cos_sim 유사도 내림차순: {ascending_cos_sim}')

  print(f'--------------------------- 최우선 순위 질문/답변 ---------------------------')
  print(f"가장 높은 코사인 유사도 idx : {int(np.argmax(cos_sim))}")

  # data에서 선택된 질문 출력
  best_sim_idx = int(np.argmax(cos_sim))
  sentence_qes = data['input'][best_sim_idx]
  print(f"선택된 질문 = {sentence_qes}")

  # data에서 선택된 질문 문장에 대한 인코딩
  selected_qes_encode = model.encode(sentence_qes)

  # 유사도 점수 측정
  # print(f'np.dot(sentence_tensor, selected_qes_encode) 값: {np.dot(sentence_tensor, sentence_encode)}')
  # print(f'(np.linalg.norm(sentence_tensor) 값: {np.linalg.norm(sentence_tensor)}')
  # print(f'(np.linalg.norm(selected_qes_encode) 값: {np.linalg.norm(selected_qes_encode)}')
  print(f'util.cos_sim 활용 코사인 유사도 : {cos_sim[0][best_sim_idx]}')
  score = np.dot(sentence_tensor, sentence_encode) / (np.linalg.norm(sentence_tensor) * np.linalg.norm(selected_qes_encode))
  print(f"직접 수식 작성한 코사인 유사도 = {score}")

  # 답변
  answer = data['output'][best_sim_idx]
  print(f"\n답변 : {answer}\n")

  # 중복 input 대응을 위해 이전 idx,sentence 저장 
  last_sim_idx_list.append(int(np.argmax(cos_sim)))
  last_sentence = sentence

  # 최우선 순위 질문 벡터 초기화 
  cos_sim[0][best_sim_idx] = 0

  print(f'--------------------------- 후순위 답변/질문 디버깅 ---------------------------')
  for i in range(1,6):
    print(f'<{i}순위 Case>')
    print(f"가장 높은 코사인 유사도 idx : {int(np.argmax(cos_sim))}")

    # data에서 선택된 질문 출력
    best_sim_idx = int(np.argmax(cos_sim))
    sentence_qes = data['input'][best_sim_idx]
    print(f"선택된 질문 = {sentence_qes}")

    # data에서 선택된 질문 문장에 대한 인코딩
    selected_qes_encode = model.encode(sentence_qes)

    # 유사도 점수 측정
    # print(f'np.dot(sentence_tensor, selected_qes_encode) 값: {np.dot(sentence_tensor, sentence_encode)}')
    # print(f'(np.linalg.norm(sentence_tensor) 값: {np.linalg.norm(sentence_tensor)}')
    # print(f'(np.linalg.norm(selected_qes_encode) 값: {np.linalg.norm(selected_qes_encode)}')
    print(f'util.cos_sim 활용 코사인 유사도 : {cos_sim[0][best_sim_idx]}')
    score = np.dot(sentence_tensor, sentence_encode) / (np.linalg.norm(sentence_tensor) * np.linalg.norm(selected_qes_encode))
    print(f"직접 수식 작성한 코사인 유사도 = {score}")

    # 답변
    answer = data['output'][best_sim_idx]
    print(f"\n답변 : {answer}\n")

    # 최우선 순위 질문 벡터 초기화 
    cos_sim[0][best_sim_idx] = 0

  print(f'-------------------------------------------------------------------------------')

  return last_sentence, last_sim_idx_list , overlap_cnt

## 3. 챗봇 실행 
## ex) talk_to_bot('내가 미래에 결혼할 수 있을까?')

In [5]:
last_sentence, last_sim_idx_list , overlap_cnt = talk_to_bot('내가 미래에 결혼할 수 있을까?')

사용자의 질문: 내가미래에결혼할수있을까? 

--------------------------- 최우선 순위 질문/답변 ---------------------------
가장 높은 코사인 유사도 idx : 5836
선택된 질문 = 내가 결혼을 할 수 있을까?
util.cos_sim 활용 코사인 유사도 : 0.7626185417175293
직접 수식 작성한 코사인 유사도 = 0.9475586414337158

답변 : 결혼에 대해서 생각이 많으신 것 같네요. 무슨 일인지 제가 들어볼 수 있을까요?

--------------------------- 후순위 답변/질문 디버깅 ---------------------------
<1순위 Case>
가장 높은 코사인 유사도 idx : 8807
선택된 질문 = 난 과연 앞으로 결혼 할 수 있을까?
util.cos_sim 활용 코사인 유사도 : 0.7526490688323975
직접 수식 작성한 코사인 유사도 = 0.9317430853843689

답변 : 결혼을 할 수 있을지 걱정이 되시는군요.

<2순위 Case>
가장 높은 코사인 유사도 idx : 7426
선택된 질문 = 나 과연 결혼은 할 수 있을까?
util.cos_sim 활용 코사인 유사도 : 0.7353234887123108
직접 수식 작성한 코사인 유사도 = 0.9603838920593262

답변 : 결혼에 대해서 고민이시군요.

<3순위 Case>
가장 높은 코사인 유사도 idx : 4870
선택된 질문 = 과연 지금 결혼하는 것이 맞을까?
util.cos_sim 활용 코사인 유사도 : 0.7037502527236938
직접 수식 작성한 코사인 유사도 = 0.9708771705627441

답변 : 결혼을 언제 할까 생각하고 계시군요.

<4순위 Case>
가장 높은 코사인 유사도 idx : 5121
선택된 질문 = 내가 결혼 생활을 잘할 수 있을까?
util.cos_sim 활용 코사인 유사도 : 0.7022172212600708
직접 수식 작성한 코사인 